In [1]:
import os
import numpy as np
#from Starter.Detection import Detector, Network
from Stochastic import Princess as SP
from Starter.AstroModel import Astromodel
from Starter import Detection


<div>
<img src="./Wiki/Princess_logo.png" width="300"/>
</div>

# Princess: Guide for a first calculation


This file goes step-by-step through the calculation of an astrophysical background starting from a CBC catalogue.
For mode details about the code structure and basic calculations please visit the README.md file.
Princess as two companioin paper which give more details on the physica behind these calculations.

This toolkit aims to be user friendly and useful to the collaboration. If you have any comments, issues or requests please contact the administrator (_caroleperigois@outlook.com_).

The calculation of the background goes through four main step, defining the four sections of this file.
1. Prepare your model
2. Prepare your detectors and Networks
3. Calculate the background
4. Analyse our background

## 1. Prepare your model: 
All population synthesis codes may have different outputs. In the next steps the code will re-build catalogues insuring that it contains all the required parameters for the next steps of calculation.
Your astrophysical model will be set in a class Princess.Astromodel and takes in entry several parameter.
* `cat_name`: (_str_) is the name you want to use for your model
* `original_cat_path`: (_str_) path to your original astrophysical catalogue
* `cat_sep`: (_str_) separator used in your original catalogue (default is tab)
* `index_column`: (_bool_) does your original file contain a columns with indexes (default is None)
* `flags`: (_dict_) this option allow to differenciate different types of CBCs is a Model. If you add this option you need to set up a dictionnary of the different categories. For example in the checks the original catalogue contain a column called 'flag' wher can be found identifiers 1 for isolated BBH and 2 for cluster BBH. Therefore the dictionnary looks like Flags = {'1': 'Iso', '2':'Cluster'}. In the next steps the code will build two catalogues out from the initial model (default is None).



In [2]:
# path_2_cat_folder contains the path to your original catalogue
path_2_cat_folder =  '/home/perigois/Documents/GreatWizardOz/'
Flags = {'1': 'Orig', '2':'Exch', '3':'Iso'}
Cat = 'Cat_stochastic_dyn_BBHs.dat'
astromodel1 = Astromodel(cat_name= 'BBH_YSCs', original_cat_path = path_2_cat_folder+Cat, 
                         cat_sep = "\t", index_column = None, flags = Flags, spin_option = 'rand_Dynamics')

If your original catalogue do not have header you can set one using the method makeHeader on your model. In the next liste are the labels allowed by the code, please note that for the masse you will need to have or the chirp mass and the mass ratio, or the two masses m1 and m2.
* Mc : Chirp mass in the source frame [Msun]
* q : mass ratio in the source frame
* m1 : mass of the first component (m1>m2) in the source frame [Msun]
* m2 : mass of the secondary component (m2<m1) in the source frame [Msun]
* Xeff : effective spin
* s1 : individual spin factor of the first component
* s2 : individual spin factor of the second component
* theta1 : angle between the first spin component and the angular momentum of the binary [rad]
* theta2 : angle between the second spin component and the angular momentum of the binary [rad] 
* a0 : semi-major axis at the formaiton of the second compact object [Rsun]
* e0 : eccentricity at the formation of the second compact object  
* inc : inclinaison angle [rad]       
* zm : redshift of merger
* zf : redshift of formation  
* Dl : luminosity distance [Mpc]
* flag : this columns cam contain a flag to differenciate sources

In [3]:
#astromodel1.makeHeader(['Mc','q','zm','zf','t_del','Z','a0','e','flag'])

Set spin option:
    True if you want to include spin in your calculations of the waveforms and background and you have the spin in your catalogue
    'Zero' if you don't want to use the spin, the code set all spins to 0.
    Model if you want Princess to generate spin values - Option available later -

In [4]:
IncAndPos = False
orbit_evo = False

Finally the MakeCat method generate the catalogue with all requiered parameters

In [5]:
astromodel1.makeCat()

print('Astromodel loaded and ready :)')

                  Mc              q              z             zf  \
count  107771.000000  107771.000000  107771.000000  107771.000000   
mean       14.538996       0.742064       2.464185       6.116513   
std         8.026325       0.203923       1.427582       4.103212   
min         4.803410       0.135198       0.044725       0.155000   
25%         6.877321       0.600000       1.473135       2.435000   
50%        13.399752       0.787523       2.170265       4.955000   
75%        21.680084       0.902443       3.147535       9.395000   
max        55.827192       1.000000      13.165935      14.995000   

              t_del              Z             a0              e  \
count  1.077710e+05  107771.000000  107771.000000  107771.000000   
mean   1.684101e+09       0.004564    2366.661568       0.342323   
std    1.939425e+09       0.004377   11075.793661       0.356084   
min    2.294068e+07       0.000200       0.032131       0.000000   
25%    3.128492e+08       0.002000    

## 2. Detectors and Networks
In this part is detailed the context of the study starting by defining the range of frequency `Freq` and the waveforms `WF_approx` to use. In this version the range has to be linear in the future specific function will be added to allow log scale, in particular for LISA band. The available waveforms are the ones define in PyCBC and the analytic one from Ajith2011. The calculation with Ajith waveforms is computationnalymore expensive and therefore not recommended.   

In [6]:
Freq = np.arange(1000)+10
WF_approx = "IMRPhenomD"

In Princess two classes has been build for this purpose in the file _Starter/Detection_ in order to define Detectors and combine them to build a Network. The Detector class takes in entry several parameters:
* `det_name`: (_str_) is the name you give to the detector
* `Pycbc`: (_bool_) True if the sensitivity is available in PyCBC, else False
* `psd_file`: (_str_) name of the sensitivity in PyCBC, or file where your sensitivity is stored
* `freq`: (_np.array_) frequency range of the study

In [7]:
H = Detection.Detector(det_name = 'H', origin = 'Pycbc', configuration = 'H', psd_file = 'aLIGODesignSensitivityP1200087', freq = Freq )
L = Detection.Detector(det_name = 'L', origin = 'Pycbc', configuration = 'L', psd_file = 'aLIGODesignSensitivityP1200087', freq = Freq )
V = Detection.Detector(det_name = 'V', origin = 'Pycbc', configuration = 'V', psd_file = 'AdVDesignSensitivityP1200087', freq = Freq )



The second class Network allow to combine different detectors to build a Network.A Network takes in entry: 
* `net_name`: (_str_) Name of the network
* `compo`: (_list of detectors_) List of the detectors in the network.
* `pic_file`: (_str_) link to the file of the power integrated curve.
* `efficiency`: (_float_) between 0 and 1 define the effective time of observation of the Network. For example during O3a, in the Hanford-Livinstone-Virgo network only 50% of the data can be used with the three pipelines. The rest of the time at least on detector pipeline was unusuable.
* `SNR_thrs`: (_int_ or _float_) Define the SNR threshold for which we assume a source is detectable
* `SNR_sub`: (_int_ or _float_) Define the SNR threshold to substract the sources. For example its commonly assumed that in HLV all source with an SNR above 8 are resolved. However for a reason of parameter uncertainty the calculation of the residual background is done by subtracting only source with a SNR above 12. 


**If only one detector is used in the study it still has to be set as a detector.**

The variable `Networks` gather all the networks used in the study.


In [8]:
HLV_Des = Detection.Network(net_name = 'HLV',compo=[H,L,V], pic_file = 'AuxiliaryFiles/PICs/Design_HLV_flow_10.txt' , efficiency = 0.5,SNR_thrs = 12 )

Networks = []

Finally the method `compute_SNR`, compute the SNR of each sources of the model, and update the catalogue(s) with a new parameter column named by `net_name` containing the SNR in the corresponding network.

In [9]:
#astromodel1.compute_SNR_opt(Networks, Freq, approx = WF_approx)

## 3. Calculate the corresponding background:

Prepare the calculation of the background with the class `Princess` :
* `freq`: (_np.array_) frequency range, preferentially the one used since the beginning. In linear scale before the LISA update
* `approx`: (_str_) waveform approximation
* `freq_ref`: (_list of float_) list of frequency of interest for the study, is usually 10Hz for 3G detectors and 25Hz for 2G.

Then the calculation is done by using the method Omega_pycbc

In [10]:
Zelda = SP.Princess(Freq, approx = WF_approx, Omega_ana_freq = [25.], Networks = [])
Zelda.Omega_pycbc(astromodel1, Networks= [])

BBH_YSCs_Orig.dat
 ***   GW COMPUTATION   ***         0/55536
           ■ ■■ ■             2%    1110/55536
          ■■■■■■■■            4%    2220/55536
          ■ ■  ■ ■            6%    3330/55536
         ■        ■           8%    4440/55536
         ■        ■           10%   5550/55536
          ■■■■ ■■■            12%   6660/55536
         ■    ■  ■■           14%   7770/55536
        ■          ■          16%   8880/55536
      ■■            ■         18%   9990/55536
 ■   ■              ■■        20%   11100/55536
  ■■■                 ■       22%   12210/55536
 ■                    ■       24%   13320/55536
  ■                  ■        26%   14430/55536
 ■         ■■     ■■  ■       28%   15540/55536
  ■       ■  ■   ■ ■  ■       30%   16650/55536
   ■      ■   ■■■ ■■ ■        32%   17760/55536
    ■■ ■■ ■  ■   ■ ■■         34%   18870/55536
    ■ ■ ■ ■  ■   ■ ■■         36%   19980/55536
   ■  ■ ■  ■ ■   ■ ■■■        38%   21090/55536
■■■■   ■ ■ ■       ■ ■        40%  

## 4. Analyse the background
This part of the code aims to extract reference values for the predicted background. Usual values are the amplitude at 10 and 25 Hz, the SNR, the number of resolved soures, the ratio of detected sources, and the ratio between residuals and total backgrouns at a reference value.

## 5. Individual analysis 
This package of the code aims to predict individual observation from LIGO-Virgo from a predefined astrophysical model. As for the background computation the user has to predefine his analysis with the _class_ 'IndividualAnalysis'. This class takes in entry the limits of the study and the assumption on GW events observations.

* `params` : (_dict_) parameters the user want to compare and the range [minimum, maximum, bins]. Default are : {'m1': [0, 100, 100], 'q': [0, 1, 20], 'zm': [0, 5, 30]}
* `iteration` : (_int_) number of iteration to extract the errors on observations
* `Network` : (Network from _class_ 'Network') network to use in the study
* `binary_type` : (_list_ of _str_) type of binaries to be compared. Default is ['BBH', 'BNS', 'NSBH'].
* `pastro_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 0.
* `SNR_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 0.
* `FAR_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 2.


In [11]:
H_O3b = Detection.Detector(det_name = 'H_O3b', configuration = 'H', origin = 'Princess', psd_file = 'Hanford_O3b', freq = Freq )
L_O3b = Detection.Detector(det_name = 'L_O3b', configuration = 'L', origin = 'Princess', psd_file = 'Livingston_O3b', freq = Freq )
V_O3b = Detection.Detector(det_name = 'V_O3b', configuration = 'V', origin = 'Princess', psd_file = 'Virgo_O3b', freq = Freq )

HLV_O3b = Detection.Network(net_name = 'HLV_O3b',compo=[H_O3b,L_O3b,V_O3b], efficiency = 0.5, SNR_thrs = 12 )


TypeError: __init__() missing 1 required positional argument: 'configuration'

In [ ]:
Analysis = IndividualAnalysis(params =  {'m1': [0, 100, 100], 'q': [0, 1, 20], 'zm': [0, 5, 30]}, Network = HLV)